# Nafigator Tutorial
This tutorial helps you to get started with the Nafigator package. We will:
1. Create a catalog file: this contains the metadata of your file. 
2. Create a naf file: this contains all the content of you text file. 
3. Access the text layer to understand the structure of NAF and do some basic search querries
4. Store your NAF files

This tutorial is set up for one pdf file. You can also import multiple files at the same time

## Getting ready

In [ ]:
#import your packages
import nafigator
import os
import stanza
import pandas as pd
from nafigator.parse2naf import generate_naf

In [ ]:
#Download and specify your (English) NLP engine
stanza.download('en')
stanza_nlp = stanza.Pipeline('en')

## 1. Create a catalog file
Cataloge files contain the meta data of your files. Here you can store the correct information
We use the [Dublin Core Metadata initiative](https://www.dublincore.org/resources/userguide/publishing_metadata/#exCon1).

In [ ]:
#create a catalog file containing your metadata
df_catalog=pd.DataFrame(columns =['dc:identifier', 'dc:source', 'dc:relation', 'dc:creator', 'dc:format', 'dc:language', 'dc:type', 'dc:coverage', 'naf:status'])

In [ ]:
doc_name = os.path.join("..", "data", "external", "naf", "annual_report_dnb_2020.naf.xml")

In [ ]:
#add the metadata of the document to the dataframe
df_catalog=nafigator.doc2catalog(doc_name, df_catalog, document_source = None, document_type="annual report", document_year= 2020)

## 2. Create a naf file
Depending on the length of your document, the creation of a NAF file containing all the relevant layers may take a while.
In this example we use "DNB's annual report"[maybe insert another file]

In [ ]:
#generate the NAF file with an input (pdf) file. The data folder contains an example for you to use, but you can specify the file you want to analyse by changing 'input'.
doc = generate_naf(input = "../data/external/naf/annual_report_dnb_2020.pdf",
                   engine = "stanza",
                   language = "en",
                   naf_version = "v3.1",
                   dtd_validation = False,
                   params = {'fileDesc': {'author': 'anonymous'}},
                   nlp = None)

## 3. Access the text layer to understand the structure of NAF and perform some basic search querries

In [ ]:
#Congratulations! You have now your first NAF file. To access the plain text of the document, run the below command.
doc.raw

When working with NAF, it's important to understand the structure of the naf.xml file.

In [ ]:
#To get information about the the layers in the NAF file, use the header function:
doc.header

As you can see, each layer is a different dictionary containing a list.
For more examples, please check: https://pypi.org/project/nafigator/

### The structure - how to use the text layer
You can use doc.text to access the following elements from the text layer:
- id: the id of the word form
- sent: sentence id of the word form
- para: paragraph id of the word form
- offset: the offset (in charachters) of the word form
- length: the length (in charachters) of the word form

In [ ]:
#By printing this result you will see the structure of the file. It is a set of dictionaries within a list.
doc.text

In [ ]:
#To access a specific part of the document, you can use lists to go through the entire file. Let's say you want to access the 4240th word in this file.
doc.text[4239]

In [ ]:
#If you want to extract specific values from the file (for example, the page number where )
doc.text[4239]["page"]

### Search
NAF makes multiple types of search possible. Here we'll show you two types:
- Exact search
- Lemmatized search

### Exact Search

In [ ]:
#The 4239th word is 'economy'. If you want to search all sentences that contain this word, you loop through the entire text file as shown below. This will show you all the sentences that contain 'economy'.
[word["sent"]for word in doc.text if word["text"]=="economy"]

### Lemmatized Search


In [ ]:
#Gather all the lemmas
lemmas = [term['lemma'] for term in doc.terms]
#Search for combined terms
economy=nafigator.sublist_indices(["economy"], lemmas)
print(economy)

### Maybe not necessary

In [ ]:
#However, sometimes you want to do lematized search instead
#first you gather the terms and words
doc_terms = {term['id']: term for term in doc.terms}
doc_words = {word['id']: word for word in doc.terms}
#Or a combination of words


### Getting more from page text

In [ ]:
# Print the text on the 22nd page
[word['text'] for word in doc.text if word['page'] == '22']

### Printing Specific Sentences

In [ ]:
#printing a sentence based on a sentence number
sentence =  doc.sentences[23]
print("Sentence: " + str(sentence["text"])+"\n")

### Get Insight in the Entities
NAF is great at storing information about the text in different layers. For example, you might want to list all the entities that are named in this document.

In [ ]:
# To get all the entities in the text, use the below command
entities=doc.entities

# If you only want to extract the organisations you can do so as well
[doc['text']for doc in doc.entities if doc['type'] == "ORG"]


## 4. Storing your NAF File
After you have generated your NAF file, you probably want to store it for later use. Especially since this is the most time consuming part of your analysis.

In [ ]:
#store you document naf file as xml 
doc.write("../data/external/naf/annual_report_dnb_2020.naf.xml")

In [ ]:
#if you want to reuse it later, import an existing naf file as shown below
doc_name = os.path.join("..", "data", "external", "naf", "annual_report_dnb_2020.naf.xml")
#doc = nafigator.NafDocument().open('notebook_data/output.naf.xml')
doc=nafigator.NafDocument().open(doc_name)